In [10]:
import sys
import matplotlib.pyplot as plt

import numpy as np
from sklearn import datasets
from sklearn import preprocessing
from sklearn import metrics
from sklearn import model_selection


%matplotlib inline

colors = ['r','b','c','g','k','m','y']
np.set_printoptions(3)

sys.path.append('../')

import Sonar.StackedAutoEncoder as SAE

# Load Data

In [3]:
dataset = datasets.load_iris()
data = dataset.data
target = dataset.target

## Select Novelty 

In [4]:
novcls = 2
nov_data = data[target==novcls]
nov_target = target[target==novcls]
#
data   = data[target != novcls]
target = target[target != novcls]

## Training Indexes 

In [24]:
# Test x Development
ntrn = 0.7
Xtrn, Xtst, Ytrn, Ytst = model_selection.train_test_split(data, target, test_size = 1.0-ntrn, stratify=target)

cv_indexes = []
kfold = model_selection.StratifiedKFold(n_splits=4)
for itrn, ival in kfold.split(Xtrn, Ytrn):
    cv_indexes.append({'itrn': itrn, 'ival': ival})

## Dataprep

In [6]:
scaler = preprocessing.StandardScaler().fit(data[itrn])
data = scaler.transform(data)
nov_data = scaler.transform(nov_data)

# Network  Configuration

In [7]:
net_config = {
    'hiddens': [data.shape[1], 10, 5, 2, 5, 10, data.shape[1]],
    'optimizers': ['adam','adam','adam'],
    'nepochs': 500,
    'batch_size': 100,
    'ninit': 10,    
    'patience' : 100,
    'itrn': itrn,
    'itst': itst,
    'ival': itst,
    'verbose': 0,
}

# Training 

In [9]:
try:
    model = SAE.StackedAutoEncoder(net_config)
    model.fit(data)
except Exception as err:
    import re, os
    x = re.match(r'C:\\Users\\txavier\\AppData\\Local\\Temp\\3\\\w*\.exe', err.__str__())
    if x is not None:
        print('Removing file')
        os.remove(err.__str__()[x.start():x.endpos])

Training 3 layer pairs
	Layer pair 1
		Structure = 4:10:4
		Activations = tanh:linear


In [ ]:
model.plot_training_curves()